In [1]:
using Revise
using PauliPropagation

Precompiling PauliPropagation
  ✓ PauliPropagation
  1 dependency successfully precompiled in 2 seconds. 10 already precompiled.


In [2]:
nq = 8

8

In [3]:
symbs = [:I for _ in 1:nq]
symbs[round(Int, nq/2)] = :Z   # as symbol. Also works but is slower.

obsint = symboltoint(symbs)  # for performance we work with bitoperations
show(obsint, nq)

IIIZIIII


In [4]:
nl = 4
topo = bricklayertopology(nq; periodic=false)
circ = hardwareefficientcircuit(nq, nl; topology=topo)
fastcirc = tofastgates(circ)
m = length(fastcirc)

124

In [5]:
using Random
Random.seed!(42)
thetas = randn(m);

In [6]:
W = Inf;                   # maximal operator weight.
min_abs_coeff = 0;          # neglect small coefficients. Only for numerical and hybrid PP.

#### Numerical Pauli Propagation
Propagates numerical coefficients.

In [7]:
@time dnum = mergingbfs(fastcirc, obsint, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops
evalagainstzero(dnum) # expectation

  0.625854 seconds (723.54 k allocations: 51.777 MiB, 3.62% gc time, 83.96% compilation time)
length(dnum) = 53247


0.21720058439757214

#### Hybrid Pauli Propagation
Propagates numerical coefficients, but can but truncated like the surrogate.

In [8]:
max_freq = Inf   # max frequency, i.e., max number of sines and cosines per path

Inf

In [9]:
@time dhyb = mergingbfs(fastcirc, obsint, NumericPathProperties(1.0), thetas; max_weight=W, max_freq=Inf, min_abs_coeff=min_abs_coeff);
@show length(dhyb)
evalagainstzero(dhyb)

  0.414006 seconds (957.78 k allocations: 53.133 MiB, 2.56% gc time, 69.54% compilation time)
length(dhyb) = 53247


0.21720058439757214

#### Pauli Propagation Surrogate
Builds a graph that can later be evaluated.

In [10]:
@time dsym = mergingbfs(circ, operatortopathdict(obsint), zeros(m); max_weight=W, max_freq=max_freq);
@show length(dsym)

final_nodes = collect(pth.coeff for (obs, pth) in zerofilter(dsym));
final_eval_node = PauliGateNode(parents=final_nodes, trig_inds=zeros(Int, length(final_nodes)), signs=ones(length(final_nodes)), param_idx=1, cummulative_value=0.0);
resetnodes(final_eval_node)
resetnodes(final_eval_node)
@time eval_list = gettraceevalorder(final_eval_node, zeros(m));
length(eval_list)  # The list of all nodes. The order is such that one can savely be evaluated after the other.

  1.392057 seconds (11.50 M allocations: 620.592 MiB, 18.93% gc time, 34.59% compilation time)
length(dsym) = 53247
  0.112288 seconds (32.47 k allocations: 5.191 MiB, 51.63% compilation time)


225500

In [11]:
@time expectation(eval_list, thetas)    # This is actually not always faster than numerical propagation, but in interesting cases it is by a lot.
                                        # making this always at least as fast is work in progress. Graph traversal is hard.

  0.096164 seconds (62.00 k allocations: 4.309 MiB, 57.30% compilation time)


0.2172005843975723